In [1]:
import numpy as np
from os import path, listdir
from os.path import isfile, join
import tensorflow as tf
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import LeaveOneOut
import cv2
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import Normalizer

from tqdm import tqdm

GENERAL_DIR = r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien" 
syn_list =  ['ADNP', 'ANKRD11', 'CDK13', 'DEAF1', 'DYRK1A', 'EHMT1', 'FBXO11','KDVS', 'SON', 'WAC', 'YY1']

In [2]:
IMAGE_SIZE = (152, 152) # set by the model
CHANNELS = 3 # RGB image
NUM_CLASSES = 8631 # classification layer will be removed
LEARN_RATE = 0.01
MOMENTUM = 0.9

DOWNLOAD_PATH = 'https://github.com/swghosh/DeepFace/releases/download/weights-vggface2-2d-aligned/VGGFace2_DeepFace_weights_val-0.9034.h5.zip'
MD5_HASH = '0b21fb70cd6901c96c19ac14c9ea8b89'

In [3]:
wt_init = tf.random_normal_initializer(mean=0, stddev=0.01)
bias_init = tf.constant_initializer(value=0.5)

In [4]:
def create_classifying_deepface(image_size=IMAGE_SIZE, channels=CHANNELS, num_classes=NUM_CLASSES, learn_rate=LEARN_RATE, momentum=MOMENTUM):
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(*image_size, channels), name='I0'))
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=11, activation=tf.nn.relu, kernel_initializer=wt_init, bias_initializer=bias_init, name='C1'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, padding='same', name='M2'))
    model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=9, activation=tf.nn.relu, kernel_initializer=wt_init, bias_initializer=bias_init, name='C3'))
    model.add(tf.keras.layers.LocallyConnected2D(filters=16, kernel_size=9, activation=tf.nn.relu, kernel_initializer=wt_init, bias_initializer=bias_init, name='L4'))
    model.add(tf.keras.layers.LocallyConnected2D(filters=16, kernel_size=7, strides=2,  activation=tf.nn.relu, kernel_initializer=wt_init, bias_initializer=bias_init, name='L5'))
    model.add(tf.keras.layers.LocallyConnected2D(filters=16, kernel_size=5, activation=tf.nn.relu, kernel_initializer=wt_init, bias_initializer=bias_init, name='L6'))
    model.add(tf.keras.layers.Flatten(name='F7'))
    model.add(tf.keras.layers.Dense(units=4096, activation=tf.nn.relu, kernel_initializer=wt_init, bias_initializer=bias_init, name='F8'))
    model.add(tf.keras.layers.Dropout(rate=0.5, name='D9'))
    model.add(tf.keras.layers.Dense(units=num_classes, activation=tf.nn.softmax, kernel_initializer=wt_init, bias_initializer=bias_init, name='F10'))

    sgd_opt = tf.keras.optimizers.SGD(learning_rate=learn_rate, momentum=momentum)
    cce_loss = tf.keras.losses.CategoricalCrossentropy()

    model.compile(optimizer=sgd_opt, loss=cce_loss, metrics=['acc'])
    weights = get_weights()
    model.load_weights(weights)

    return model

In [5]:
def get_weights():
    filename = 'deepface.zip'
    downloaded_file_path = tf.keras.utils.get_file(filename, DOWNLOAD_PATH,
        md5_hash=MD5_HASH, extract=True)
    downloaded_h5_file = path.join(path.dirname(downloaded_file_path),
        path.basename(DOWNLOAD_PATH).rstrip('.zip'))
    return downloaded_h5_file

In [6]:
model  = create_classifying_deepface()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
C1 (Conv2D)                  (None, 142, 142, 32)      11648     
_________________________________________________________________
M2 (MaxPooling2D)            (None, 71, 71, 32)        0         
_________________________________________________________________
C3 (Conv2D)                  (None, 63, 63, 16)        41488     
_________________________________________________________________
L4 (LocallyConnected2D)      (None, 55, 55, 16)        62774800  
_________________________________________________________________
L5 (LocallyConnected2D)      (None, 25, 25, 16)        7850000   
_________________________________________________________________
L6 (LocallyConnected2D)      (None, 21, 21, 16)        2829456   
_________________________________________________________________
F7 (Flatten)                 (None, 7056)              0

In [7]:
from tqdm import tqdm

GENERAL_DIR = r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien" 
syn_list =  ['ADNP', 'ANKRD11', 'CDK13', 'DEAF1', 'DYRK1A', 'EHMT1', 'FBXO11','KDVS', 'SON', 'WAC', 'YY1']

In [8]:
# average heatmap per syndrome
layers = ["C3", "L4", "L5", "L6"]
img_name = ["patient", "control"]


rows, columns = 4, 2

for syn in syn_list:
    directory = "heatmaps/{}".format(syn)
    if not os.path.exists(directory):
        os.makedirs(directory)
    path_patient = GENERAL_DIR + "//{}//{}-patients".format(syn, syn)
    path_control = GENERAL_DIR + "//{}//{}-selected-ID-controls".format(syn, syn)

    patients = [f for f in listdir(path_patient) if ".jpg" in f]
    controls = [f for f in listdir(path_control) if ".jpg" in f]
    fig = plt.figure(figsize=(18, 18))
    
    for l, name in enumerate(layers):

        # all patients
        conv_layer = model.get_layer(name)
        heatmap_model = tf.keras.models.Model([model.inputs], [conv_layer.output, model.output])

        all_heatmaps = []
        for p in patients:
            img = cv2.imread(join(path_patient, p))
            img = cv2.resize(img, (IMAGE_SIZE))               
            img_tensor = np.expand_dims(img, axis=0)
            
            conv_output, predictions = heatmap_model(img_tensor)

            heatmap = np.squeeze(conv_output.numpy())
            heatmap = np.sum(heatmap, axis=2)
            
            all_heatmaps.append(heatmap)
                    
        all_heatmaps = np.array(all_heatmaps)
        all_heatmaps = np.mean(all_heatmaps, axis=0)
        
        nr = l * 2 + 1 
        fig.add_subplot(rows, columns, nr)
        plt.imshow(all_heatmaps)
        plt.title("Heatmap of layer {}".format(name))
        
        
        # all controls
        conv_layer = model.get_layer(name)
        heatmap_model = tf.keras.models.Model([model.inputs], [conv_layer.output, model.output])

        all_heatmaps = []
        for p in controls:
            img = cv2.imread(join(path_control, p))
            img = cv2.resize(img, (IMAGE_SIZE))               
            img_tensor = np.expand_dims(img, axis=0)
            
            conv_output, predictions = heatmap_model(img_tensor)

            heatmap = np.squeeze(conv_output.numpy())
            heatmap = np.sum(heatmap, axis=2)
            
            all_heatmaps.append(heatmap)
                    
        all_heatmaps = np.array(all_heatmaps)
        all_heatmaps = np.mean(all_heatmaps, axis=0)
        
        nr = l * 2 + 1 + 1
        fig.add_subplot(rows, columns, nr)
        plt.imshow(all_heatmaps)
        plt.title("Heatmap of layer {} control".format(name))
        

    plt.savefig('heatmaps/{}/avg-{}.png'.format(syn, syn), bbox_inches='tight')
    plt.close()
        